# Project Experiments

In this notebook, we will be presenting the code for the experiments that have been done in the project. These experiments are namely:

- Check test predictions for trained model
- Generate predictions for ImageNet test dataset
- Conduct ablation study for model with 2 different architectures

These experiments have mostly been done on Ada via scripts, but have been compiled here in this notebook as proof of work. All the generated results have been uploaded to the OneDrive link shared, and the code present here is equivalent to the code used in the notebooks.

The python script train.py was run on Ada to train the models and generate checkpoints.

### Sample colourization output

In this section, we will take a random subset of images and attempt to output their colourization.